In [1]:
import pandas as pd
from collections import Counter
import ast
from itertools import chain
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data=pd.read_csv("wuzzuf.csv")

In [34]:
data=data.drop_duplicates()

In [35]:
data["Number_of_Applicants"]=data["Number_of_Applicants"].replace('Be the First to Apply',1)

In [36]:
data["Number_of_Positions"] = data["Number_of_Positions"].fillna("1")
data["Number_of_Positions"] = data["Number_of_Positions"].astype(str)
data["Number_of_Positions"] = (data["Number_of_Positions"].str.replace(",", "").str.split().str[0].astype("Int64"))

In [ ]:
all_skills = [skill for skills_list in data["Skills"] for skill in skills_list]
skill_frequency = Counter(all_skills)
top_200_skills = skill_frequency.most_common(200)
top_200_skill_names = [skill for skill, _ in top_200_skills]
print(top_200_skill_names)

In [3]:
standardization_map = {
    "ms office": "microsoft office",
    "ms excel": "microsoft excel",
    "erp": "enterprise resource planning",
    "crm": "customer relationship management",
    "seo": "search engine optimization",
    "hr": "human resources",
    "b2b": "business-to-business",
    "pmp": "project management professional",
    "qa": "quality assurance",
    "it": "information technology",
    "sql": "structured query language",
    "cpa": "certified public accountant",
    "cma": "certified management accountant",
    "hvac": "heating, ventilation, and air conditioning",
    "fmcg": "fast-moving consumer goods",
    "bim": "building information modeling",
    "cad": "computer-aided design"
}

words_to_remove = {"skills", "experience", "field", "technical", "office", "management", "assistant"}
def clean_skill(skill):
    skill = skill.lower().strip()
    skill = standardization_map.get(skill, skill)  
    cleaned_words = [word for word in skill.split() if word not in words_to_remove]
    return " ".join(cleaned_words) if cleaned_words else None 

def process_skills(skills):
    if isinstance(skills, str):
        try:
            skills_list = ast.literal_eval(skills)  
            if not isinstance(skills_list, list):
                return []  
        except (SyntaxError, ValueError):
            return []
    elif isinstance(skills, list):
        skills_list = skills
    else:
        return []
    cleaned_skills = [clean_skill(skill) for skill in skills_list]
    return [skill for skill in cleaned_skills if skill] 
data = data.dropna(subset=["Skills"])
data["Skills"] = data["Skills"].apply(process_skills)